# 🌱 Vertical Farming Yield Forecasting with XGBoost

This notebook demonstrates a "Golden Path" for predicting leafy-green harvest yield in a multi-layer vertical farm using modern ensemble methods (XGBoost) and explainability (SHAP). Accurate yield forecasts are critical for profitability, harvest planning, labor scheduling, and reducing food waste in controlled environment agriculture (CEA). We'll:

- Simulate a realistic 18-month IoT dataset (6,000 rows) with daily/weekly/seasonal signals
- Engineer features (rolling windows, time features)
- Compare a baseline model (Random Forest) with XGBoost
- Perform hyperparameter tuning and evaluate using MAE, RMSE, R², and MAPE
- Explain model drivers with SHAP and actionable business insights
- Export the trained model for production use

**Author:** Senior AI Architect — Ambient Systems
**Scenario:** Vertical Farm Yield Prediction — Predict harvest weight (g) of leafy greens using sensor data (Light, Temp, Humidity, CO₂, EC, pH, Plant Age)

In [ ]:
# Install required packages (uncomment to run if packages are not present)
# Use quiet installs to keep notebook outputs clean

!pip install -q xgboost shap joblib scikit-learn matplotlib seaborn pandas numpy


In [ ]:
# Standard imports and reproducible settings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.model_selection import TimeSeriesSplit, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
import xgboost as xgb

# For SHAP explainability
import shap

# Plot style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [ ]:
# ------------------------------
# Data Simulation — realistic 18-month dataset (6,000 rows)
# ------------------------------

def simulate_vertical_farm_data(n_rows=6000, start_date='2024-07-01', seed=RANDOM_STATE):
    np.random.seed(seed)

    # Create a time index spanning ~18 months
    start = pd.to_datetime(start_date)
    end = start + pd.DateOffset(months=18)

    # Create timestamps evenly spaced (n_rows points)
    timestamps = pd.date_range(start, end, periods=n_rows)
    df = pd.DataFrame({'timestamp': timestamps})

    # Time features: hour, day, month, dayofweek
    df['hour'] = df['timestamp'].dt.hour + df['timestamp'].dt.minute / 60
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['month'] = df['timestamp'].dt.month

    # Seasonal yearly component (long wave)
    days = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds() / (3600 * 24)
    yearly = 10 * np.sin(2 * np.pi * days / 365.25)  # scale

    # Weekly cycle
    weekly = 2.5 * np.sin(2 * np.pi * days / 7)

    # Daily cycle (strong for light)
    daily = 30 * np.sin(2 * np.pi * df['hour'] / 24 - 0.5)  # phase shift

    # Light (umol m^-2 s^-1): baseline + daily + seasonal + noise
    df['Light'] = np.clip(200 + daily + yearly * 5 + np.random.normal(0, 15, n_rows), 50, 800)

    # Temperature (C): controlled but with small diurnal and seasonal shifts
    df['Temp'] = np.clip(20 + 2 * np.sin(2 * np.pi * df['hour'] / 24) + 1.5 * yearly / 10 + np.random.normal(0, 0.8, n_rows), 15, 28)

    # Humidity (%): inverse of temperature to some extent, plus noise and weekly patterns
    df['Humidity'] = np.clip(60 - 4 * np.sin(2 * np.pi * df['hour'] / 24) + 3 * np.cos(2 * np.pi * df['dayofweek'] / 7) + np.random.normal(0, 4, n_rows), 30, 95)

    # CO2 (ppm): controlled enrichment cycles with periodic boosts
    df['CO2'] = np.clip(400 + 60 * (0.5 + 0.5 * np.sign(np.sin(2 * np.pi * df['hour'] / 24))) + 10 * weekly + np.random.normal(0, 15, n_rows), 350, 1200)

    # EC (mS/cm): nutrient concentration with mild drift and offsets
    df['EC'] = np.clip(1.8 + 0.2 * np.sin(days / 30) + np.random.normal(0, 0.05, n_rows), 1.2, 2.6)

    # pH: kept near neutral with small fluctuations
    df['pH'] = np.clip(5.8 + 0.2 * np.sin(days / 90) + np.random.normal(0, 0.05, n_rows), 5.4, 6.6)

    # Plant Age (days since seeding), cyclic per harvest cycle (e.g., 28-day cycles) with some variation
    cycle_length = 28
    df['Plant_Age'] = ((days % cycle_length) + np.random.normal(0, 1, n_rows)).clip(0, cycle_length)

    # Growth Stage categorical from Plant_Age
    bins = [0, 7, 14, 21, 28]
    labels = ['G0', 'G1', 'G2', 'G3']
    df['Growth_Stage'] = pd.cut(df['Plant_Age'], bins=bins, labels=labels, include_lowest=True)

    # Base yield influenced by age and light, temperature near optimal ~21°C is best
    temp_effect = -0.5 * (df['Temp'] - 21) ** 2 + 4  # parabolic peak at 21C
    light_effect = 0.01 * df['Light']
    co2_effect = 0.003 * (df['CO2'] - 400)
    ec_effect = 1.5 * (df['EC'] - 1.8)
    ph_effect = -0.8 * (df['pH'] - 5.9) ** 2 + 0.5

    # Build target: future harvest weight per plant (g) with noise
    base_yield = (0.5 * df['Plant_Age'] + temp_effect + light_effect + co2_effect + ec_effect + ph_effect)
    noise = np.random.normal(0, 3, n_rows)

    # Add seasonal yield improvement and occasional bump events (management improvements)
    management_trend = 0.02 * days
    bump = np.where(np.random.rand(n_rows) < 0.005, np.random.uniform(5, 15, n_rows), 0)

    df['Yield_g'] = (np.clip(base_yield + management_trend + bump + noise, 5, 150)).round(2)

    return df

# Generate dataset
df = simulate_vertical_farm_data()

# Basic sanity checks
print(f"Rows: {len(df)} | Date Range: {df['timestamp'].min().date()} -> {df['timestamp'].max().date()}")
df.head()

In [ ]:
# Quick data check and descriptive plots

# Convert timestamp to index for plotting convenience
df_plot = df.set_index('timestamp').resample('D').mean()

# Historical yield trend (daily average)
plt.figure(figsize=(14, 5))
plt.plot(df_plot.index, df_plot['Yield_g'], label='Daily Avg Yield (g)')
plt.title('Historical Daily Average Yield (g) — Simulated')
plt.xlabel('Date')
plt.ylabel('Yield (g)')
plt.legend()
plt.show()

# Show distribution of core features
df[['Yield_g', 'Light', 'Temp', 'Humidity', 'CO2', 'EC', 'pH', 'Plant_Age']].describe().T

In [ ]:
# Feature engineering: rolling window summaries and time features

# We'll compute short-term aggregates that an operator or model would reasonably use
rolling_hours = 24  # 24-hour mean roughly

# Create rolling stats on a resampled hourly dataframe to keep memory reasonable
df_hourly = df.set_index('timestamp').resample('H').mean().interpolate()

for col in ['Light', 'Temp', 'Humidity', 'CO2', 'EC', 'pH']:
    df_hourly[f'{col}_24h_mean'] = df_hourly[col].rolling(window=rolling_hours, min_periods=1).mean()
    df_hourly[f'{col}_24h_std'] = df_hourly[col].rolling(window=rolling_hours, min_periods=1).std().fillna(0)

# Bring Plant_Age forward as-is and add time features
df_hourly['Plant_Age'] = df_hourly['Plant_Age'].fillna(method='ffill')

# Reset index and merge with original (downsample original timestamps to hourly mean)
df_features = df_hourly.reset_index()

# We'll sample from the hourly features back to our original timestamps (nearest)
df_merged = pd.merge_asof(df.sort_values('timestamp'), df_features, on='timestamp', direction='nearest', suffixes=(None, '_hr'))

# Derive simple time-based features
df_merged['hour'] = df_merged['timestamp'].dt.hour
df_merged['dayofweek'] = df_merged['timestamp'].dt.dayofweek

# Drop any duplicate columns and confirm size
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]
print('Feature set shape:', df_merged.shape)

df_merged.head()

In [ ]:
# ------------------------------
# Modeling: Train/Test split, baseline model, and metrics
# ------------------------------

# Choose features for modeling (exclude timestamp and categorical raw)
feature_cols = [
    'Light', 'Temp', 'Humidity', 'CO2', 'EC', 'pH', 'Plant_Age',
    'Light_24h_mean', 'Temp_24h_mean', 'Humidity_24h_mean', 'CO2_24h_mean', 'EC_24h_mean', 'pH_24h_mean',
    'Light_24h_std', 'Temp_24h_std'
]

# Create dataset and drop missing
model_df = df_merged.dropna(subset=feature_cols + ['Yield_g']).copy()

# Time-based split: last 20% as hold-out
split_idx = int(len(model_df) * 0.8)
train = model_df.iloc[:split_idx]
test = model_df.iloc[split_idx:]

X_train = train[feature_cols]
y_train = train['Yield_g']
X_test = test[feature_cols]
y_test = test['Yield_g']

# Scaling features for linear baseline
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Baseline: Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X_train, y_train)

# XGBoost baseline (quick)
xgb_base = xgb.XGBRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1, tree_method='hist')
xgb_base.fit(X_train, y_train)

# Evaluation helper
from math import sqrt

def evaluate_model(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print(f"{name} — MAE: {mae:.3f} g | RMSE: {rmse:.3f} g | R²: {r2:.3f} | MAPE: {mape:.2f}%")
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}

# Baseline evaluations
rf_preds = rf.predict(X_test)
xgb_preds = xgb_base.predict(X_test)

rf_metrics = evaluate_model('Random Forest', y_test, rf_preds)
xgb_base_metrics = evaluate_model('XGBoost (base)', y_test, xgb_preds)

In [ ]:
# ------------------------------
# Hyperparameter tuning for XGBoost (RandomizedSearchCV)
# ------------------------------

param_dist = {
    'n_estimators': [100, 200, 400],
    'max_depth': [3, 4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 2, 4]
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='hist', random_state=RANDOM_STATE, n_jobs=-1)

# TimeSeriesSplit for CV to respect temporal order
tscv = TimeSeriesSplit(n_splits=4)

rand_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=25,
    scoring='neg_mean_absolute_error',
    cv=tscv,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=1
)

rand_search.fit(X_train, y_train)
print('Best params:', rand_search.best_params_)

best_xgb = rand_search.best_estimator_
best_preds = best_xgb.predict(X_test)
best_metrics = evaluate_model('XGBoost (tuned)', y_test, best_preds)

In [ ]:
# ------------------------------
# Compare metrics and visualize Actual vs Predicted
# ------------------------------

import pandas as _pd

metrics_df = _pd.DataFrame([
    {**{'Model': 'Random Forest'}, **rf_metrics},
    {**{'Model': 'XGBoost (base)'}, **xgb_base_metrics},
    {**{'Model': 'XGBoost (tuned)'}, **best_metrics}
])
metrics_df.set_index('Model', inplace=True)
metrics_df.style.format({"MAE": "{:.2f}", "RMSE": "{:.2f}", "R2": "{:.3f}", "MAPE": "{:.2f}%"})

# Actual vs Predicted scatter for tuned XGBoost
plt.figure(figsize=(7, 7))
plt.scatter(y_test, best_preds, alpha=0.4)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Yield (g)')
plt.ylabel('Predicted Yield (g)')
plt.title(f'Actual vs Predicted — XGBoost (R² = {best_metrics["R2"]:.3f})')
plt.show()

In [ ]:
# ------------------------------
# Feature importance (bar chart) and SHAP explainability
# ------------------------------

# Feature importance from XGBoost
import numpy as _np
fi = best_xgb.feature_importances_
fi_df = pd.DataFrame({'feature': feature_cols, 'importance': fi}).sort_values('importance', ascending=False)

# Top 8 features
top_n = fi_df.head(8)
plt.figure(figsize=(10, 5))
sns.barplot(data=top_n, x='importance', y='feature', palette='viridis')
plt.title('Top 8 Feature Importances — XGBoost')
plt.xlabel('Importance')
plt.show()

# SHAP explanation (TreeExplainer is efficient for XGBoost)
print('Computing SHAP values (may take a moment)...')
explainer = shap.TreeExplainer(best_xgb)
shap_values = explainer.shap_values(X_train)

# SHAP summary plot
shap.initjs()
shap.summary_plot(shap_values, X_train, plot_type='bar', show=True, max_display=12)

# More detailed SHAP summary
shap.summary_plot(shap_values, X_train, show=True, max_display=12)

In [ ]:
# ------------------------------
# Business-friendly interpretation of evaluation metrics
# ------------------------------

print('Metric interpretations:')
print('- MAE (Mean Absolute Error): average error in grams (e.g., MAE=3g means predictions off by ~3g on average)')
print('- RMSE (Root Mean Squared Error): penalizes larger mistakes; useful when large errors are costly')
print('- R²: proportion of variance explained; closer to 1 is better')
print('- MAPE: percentage error (e.g., MAPE=8% → predictions typically within ±8%)')

print('\nModel summary:')
print(metrics_df)


In [ ]:
# ------------------------------
# Save model and scaler for production
# ------------------------------

model_path = 'xgb_yield_model.joblib'
scaler_path = 'scaler.joblib'

# Save
joblib.dump(best_xgb, model_path)
joblib.dump(scaler, scaler_path)
print('Saved model to', model_path)
print('Saved scaler to', scaler_path)

# Example: how to make a real-time prediction on a new (single) sensor reading
sample = X_test.iloc[0].to_frame().T  # single sample
print('\nSample input:')
print(sample)

# Predict with model (no scaler required because XGBoost used original feature space)
pred = best_xgb.predict(sample)
print(f'Predicted yield (g) for sample: {pred[0]:.2f} g')